## Read Data

In [1]:
import pandas as pd
import numpy as np

# for reg model training
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

# for random forest
from sklearn.ensemble import RandomForestClassifier

# for multi label classification
from sklearn.datasets import make_multilabel_classification
from skmultilearn.problem_transform import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.problem_transform import BinaryRelevance

# for evaluation metrics
%run -i helper_functions.py

import warnings
warnings.filterwarnings("ignore")

In [3]:
selected_train = pd.read_csv('Data/selected_train.csv')
selected_test = pd.read_csv('Data/selected_test.csv')

In [4]:
selected_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,none,min_length_scaled,num_words_vs_length,exclamation_marks_vs_length,...,105,114,132,135,139,143,156,157,170,198
0,0,0,0,0,0,0,1,0.009393,0.181132,0.000000,...,-0.091320,-0.017755,0.003997,-0.199211,-0.109984,0.089084,0.012651,-0.019501,-0.091946,-0.113697
1,0,0,0,0,0,0,1,0.000723,0.160714,0.008929,...,-0.118445,0.039541,0.017522,0.001397,0.000094,0.021135,0.039308,-0.067295,-0.079366,-0.044959
2,0,0,0,0,0,0,1,0.007225,0.188841,0.000000,...,-0.131555,-0.060714,0.026459,-0.029582,-0.146134,0.109907,0.036539,-0.053628,0.027530,-0.211230
3,0,0,0,0,0,0,1,0.007948,0.175719,0.000000,...,-0.145382,-0.004285,-0.000706,-0.134984,-0.214832,0.177560,0.007675,-0.087473,0.106848,-0.015987
4,0,0,0,0,0,0,1,0.003613,0.208955,0.000000,...,0.007073,-0.204627,0.162032,0.013798,-0.221076,0.163578,-0.068310,-0.031184,-0.017280,0.038566


In [5]:
selected_train.shape

(159571, 48)

In [6]:
selected_test.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,min_length_scaled,num_words_vs_length,exclamation_marks_vs_length,num_unique_words_scaled,...,114,132,135,139,143,156,157,170,198,none
0,0,0,0,0,0,0,0.002890,0.166667,0.000000,0.032911,...,0.040738,-0.010083,-0.158238,-0.204194,0.116565,0.098296,-0.206160,0.046779,-0.036748,1
1,0,0,0,0,0,0,0.005058,0.187500,0.000000,0.015190,...,-0.013467,0.003550,-0.117174,-0.075475,0.043945,-0.148617,-0.350740,0.113815,-0.070907,1
2,0,0,0,0,0,0,0.000723,0.149554,0.002232,0.118987,...,-0.125080,0.045087,-0.119656,-0.274707,0.102546,0.100782,-0.107205,-0.050780,-0.098343,1
3,0,0,0,0,0,0,0.007225,0.169661,0.000000,0.070886,...,-0.023304,-0.005002,-0.281848,-0.157011,0.184323,0.067625,-0.066242,0.023737,-0.111269,1
4,0,0,0,0,0,0,0.003613,0.167665,0.000000,0.060759,...,0.055657,0.024936,-0.181419,-0.165707,0.194511,0.143557,-0.059017,0.136676,-0.004530,1


In [7]:
selected_test.shape

(63978, 48)

In [8]:
selected_test.columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'min_length_scaled', 'num_words_vs_length',
       'exclamation_marks_vs_length', 'num_unique_words_scaled',
       'verbs_vs_length', 'num_uppercase_scaled', 'uppercase_vs_length',
       'sentiment', 'bad_toxic_vs_length', 'bad_severe_toxic_vs_length',
       'bad_obscene_vs_length', 'bad_threat_vs_length', 'bad_insult_vs_length',
       'bad_identity_hate_vs_length', '29', '34', '46', '47', '53', '54', '65',
       '72', '82', '86', '87', '93', '95', '96', '98', '100', '103', '105',
       '114', '132', '135', '139', '143', '156', '157', '170', '198', 'none'],
      dtype='object')

In [9]:
features = selected_test.columns[7:-1]
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

## Model Building

### Logistic Regression Using OneVsRest

In [117]:
# with reweight
classifier_log_ovr = MultiOutputClassifier(LogisticRegression(max_iter=10000, class_weight='balanced'))
# train
classifier_log_ovr.fit(selected_train[features], selected_train[labels])
# predict
predictions_log_ovr = classifier_log_ovr.predict(selected_train[features])

In [118]:
# predict probability
predictions_proba_log_ovr = classifier_log_ovr.predict_proba(selected_train[features])

In [191]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_log_ovr)

Accuracy score:  0.9071573155523247
Precision score:  0.7835665408904706
Recall score:  0.4539005071514046
F1 score:  0.5705580253670303


In [201]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions_log_ovr[:,i], np.array(predictions_proba_log_ovr[i][:,1]).T)
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.9446954647147665
Precision score:  0.8416605049117989
Recall score:  0.5209886229894076
F1 score:  0.643592746658051
Confusion matrix:
[[142778   1499]
 [  7326   7968]]
Logarithmic Loss:  0.15238669789000686
ROC AUC score:  0.941303466965538


fitting for label severe_toxic
Accuracy score:  0.990236321136046
Precision score:  0.5300813008130081
Recall score:  0.2043887147335423
F1 score:  0.2950226244343891
Confusion matrix:
[[157687    289]
 [  1269    326]]
Logarithmic Loss:  0.029947837509383204
ROC AUC score:  0.9689319896693167


fitting for label obscene
Accuracy score:  0.9676883644271203
Precision score:  0.8245614035087719
Recall score:  0.49508817611551664
F1 score:  0.61869545925159
Confusion matrix:
[[150232    890]
 [  4266   4183]]
Logarithmic Loss:  0.09489150128336714
ROC AUC score:  0.9569661176088267


fitting for label threat
Accuracy score:  0.996973134216117
Precision score:  0.41379310344827586
Recall score:  0.025104602

In [230]:
# using cross validation
# with reweight
classifier_log_ovr_cv = MultiOutputClassifier(LogisticRegressionCV(max_iter=10000, class_weight='balanced',cv = 5))
# train
classifier_log_ovr_cv.fit(selected_train[features], selected_train[labels])
# predict
predictions_log_ovr_cv = classifier_log_ovr.predict(selected_train[features])

In [231]:
# predict probability
predictions_proba_log_ovr_cv = classifier_log_ovr_cv.predict_proba(selected_train[features])

In [232]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_log_ovr_cv)

Accuracy score:  0.9071573155523247
Precision score:  0.7835665408904706
Recall score:  0.4539005071514046
F1 score:  0.5705580253670303


The F1 score does not have obvious improve, hence wouldn't use cross validation for logistic regression for saving CPU

### Logistic Regression Using Binary Relevance

In [111]:
classifier_log_br = BinaryRelevance(LogisticRegression(max_iter=10000, class_weight='balanced'))
# train
classifier_log_br.fit(selected_train[features], selected_train[labels])
# predict
predictions_log_br = classifier_log_br.predict(selected_train[features])

In [112]:
# predict probability
predictions_proba_log_br = classifier_log_br.predict_proba(selected_train[features])

In [204]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_log_br)

Accuracy score:  0.7823790037036805
Precision score:  0.3823137948352328
Recall score:  0.8699071172146561
F1 score:  0.5202510172396128


In [227]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions_log_br[:,i].toarray(), predictions_proba_log_br[:,i].toarray())
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.8878743631361589
Precision score:  0.4545263600084016
Recall score:  0.8489603766182817
F1 score:  0.5920656634746921
Confusion matrix:
[[128695  15582]
 [  2310  12984]]
Logarithmic Loss:  0.2976558490655549
ROC AUC score:  0.9450675196913704


fitting for label severe_toxic
Accuracy score:  0.9482738091507855
Precision score:  0.15543826968850252
Recall score:  0.941692789968652
F1 score:  0.26683247468466864
Confusion matrix:
[[149815   8161]
 [    93   1502]]
Logarithmic Loss:  0.20756119824661315
ROC AUC score:  0.9819963287943583


fitting for label obscene
Accuracy score:  0.9228180559124152
Precision score:  0.39690749133564385
Recall score:  0.8810510119540774
F1 score:  0.547272459932363
Confusion matrix:
[[139811  11311]
 [  1005   7444]]
Logarithmic Loss:  0.2478629532743535
ROC AUC score:  0.9652468745132916


fitting for label threat
Accuracy score:  0.9027517531381015
Precision score:  0.02757884156301043
Recall score:  0.918410

### Logistic Regression Using ClassifierChain

In [124]:
classifier_log_chain = ClassifierChain(LogisticRegression(max_iter=10000, class_weight='balanced'))
# train
classifier_log_chain.fit(selected_train[features], selected_train[labels])
# predict
predictions_log_chain = classifier_log_chain.predict(selected_train[features])

In [125]:
# predict probability
predictions_proba_log_chain = classifier_log_chain.predict_proba(selected_train[features])

In [234]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_log_chain)

Accuracy score:  0.7989922980992787
Precision score:  0.3265042142598098
Recall score:  0.8962903869166334
F1 score:  0.4593230547737794


In [235]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions_log_chain[:,i].toarray(), predictions_proba_log_chain[:,i].toarray())
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.8878743631361589
Precision score:  0.4545263600084016
Recall score:  0.8489603766182817
F1 score:  0.5920656634746921
Confusion matrix:
[[128695  15582]
 [  2310  12984]]
Logarithmic Loss:  0.2976558490655549
ROC AUC score:  0.9450675196913704


fitting for label severe_toxic
Accuracy score:  0.8495403300098389
Precision score:  0.06133552528573665
Recall score:  0.9824451410658307
F1 score:  0.11546255019710422
Confusion matrix:
[[133995  23981]
 [    28   1567]]
Logarithmic Loss:  0.2573653047966825
ROC AUC score:  0.9804797379642446


fitting for label obscene
Accuracy score:  0.8663353616885273
Precision score:  0.27425687044307345
Recall score:  0.9260267487276601
F1 score:  0.4231819779863158
Confusion matrix:
[[130418  20704]
 [   625   7824]]
Logarithmic Loss:  0.540121157439039
ROC AUC score:  0.9608222001382551


fitting for label threat
Accuracy score:  0.8502798127479304
Precision score:  0.018151883103519242
Recall score:  0.92259

### Logistic Regression Using LabelPowerSet

In [132]:
classifier_log_lps = LabelPowerset(LogisticRegression(max_iter=10000, class_weight='balanced'))
# train
classifier_log_lps.fit(selected_train[features], selected_train[labels])
# predict
predictions_log_lps = classifier_log_lps.predict(selected_train[features])

In [133]:
# predict probability
predictions_proba_log_lps = classifier_log_lps.predict_proba(selected_train[features])

In [237]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_log_lps)

Accuracy score:  0.5402924090216894
Precision score:  0.2638933873790623
Recall score:  0.6392102114080574
F1 score:  0.35461867971907374


In [238]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions_log_lps[:,i].toarray(), predictions_proba_log_lps[:,i].toarray())
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.8733792481089923
Precision score:  0.39722931402502826
Recall score:  0.6205701582319864
F1 score:  0.48439533518768985
Confusion matrix:
[[129875  14402]
 [  5803   9491]]
Logarithmic Loss:  0.3694693161758654
ROC AUC score:  0.9004981145332336


fitting for label severe_toxic
Accuracy score:  0.9586641683012578
Precision score:  0.1504263945197819
Recall score:  0.6746081504702194
F1 score:  0.24599908550525834
Confusion matrix:
[[151899   6077]
 [   519   1076]]
Logarithmic Loss:  0.08433989644240165
ROC AUC score:  0.967695061175913


fitting for label obscene
Accuracy score:  0.8141015598072332
Precision score:  0.17512480475330006
Recall score:  0.6767664812403835
F1 score:  0.2782481751824818
Confusion matrix:
[[124189  26933]
 [  2731   5718]]
Logarithmic Loss:  0.39331131867579655
ROC AUC score:  0.8822640557964808


fitting for label threat
Accuracy score:  0.8439879426712874
Precision score:  0.014900464894504709
Recall score:  0.78

### Random Forest Using Power Set Labelling

In [39]:
rf = RandomForestClassifier(random_state = 0,class_weight='balanced',n_estimators=100,max_depth = 18)
rf.fit(selected_train[features], selected_train[labels])
predictions_rf = rf.predict(selected_train[features])

In [40]:
# predict probability
predictions_proba_rf = rf.predict_proba(selected_train[features])

In [41]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_rf)

Accuracy score:  0.902219074894561
Precision score:  0.6087704906507851
Recall score:  0.9897430052994473
F1 score:  0.7518977898416708


In [56]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions_rf[:,i], np.array(predictions_proba_rf[i][:,1]).T)
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.9194590495766775
Precision score:  0.5442006950477846
Recall score:  0.9829344841114163
F1 score:  0.7005452257793934
Confusion matrix:
[[131686  12591]
 [   261  15033]]
Logarithmic Loss:  0.33269481489117914
ROC AUC score:  0.9935500585637245


fitting for label severe_toxic
Accuracy score:  0.9967851301301615
Precision score:  0.7573739295908658
Recall score:  0.9981191222570532
F1 score:  0.8612388423045713
Confusion matrix:
[[157466    510]
 [     3   1592]]
Logarithmic Loss:  0.030561200725369837
ROC AUC score:  0.9995744324005884


fitting for label obscene
Accuracy score:  0.9740053017152239
Precision score:  0.6717240277888685
Recall score:  0.9956207835246774
F1 score:  0.8022124737745566
Confusion matrix:
[[147011   4111]
 [    37   8412]]
Logarithmic Loss:  0.22906372735169245
ROC AUC score:  0.9992421914677494


fitting for label threat
Accuracy score:  0.9985398349324125
Precision score:  0.6772793053545586
Recall score:  0.97907

In [42]:
# on test data
predictions_rf_test = rf.predict(selected_test[features])
predictions_proba_rf_test = rf.predict_proba(selected_test[features])

In [44]:
# model evaluation on test data
get_overall_evaluation_score(selected_test[labels], predictions_rf_test)

Accuracy score:  0.7988371002532121
Precision score:  0.38103330011664166
Recall score:  0.7338943302524487
F1 score:  0.4920798987105066


### Random Forest Using ClassificationChain

In [60]:
classifier_chain = ClassifierChain(RandomForestClassifier(random_state = 0,max_depth = 18))
# train
classifier_chain.fit(selected_train[features], selected_train[labels])
# predict
predictions_chain = classifier_chain.predict(selected_train[features])

In [61]:
# predict probability
predictions_proba_chain = classifier_chain.predict_proba(selected_train[features])

In [62]:
# model evaluation overall score
get_overall_evaluation_score(selected_train[labels], predictions_chain)

Accuracy score:  0.9778217846601199
Precision score:  0.9943418659814655
Recall score:  0.8716166163314149
F1 score:  0.9283244493707036


In [66]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions_chain[:,i].toarray(), predictions_proba_chain[:,i].toarray())
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.9836812453390654
Precision score:  0.9998424452497243
Recall score:  0.829867922060939
F1 score:  0.9069601257681864
Confusion matrix:
[[144275      2]
 [  2602  12692]]
Logarithmic Loss:  0.055889047561944016
ROC AUC score:  0.9973127426510526


fitting for label severe_toxic
Accuracy score:  0.9993482525020211
Precision score:  1.0
Recall score:  0.9347962382445141
F1 score:  0.966299416720674
Confusion matrix:
[[157976      0]
 [   104   1491]]
Logarithmic Loss:  0.00719728881320667
ROC AUC score:  0.9996034475614962


fitting for label obscene
Accuracy score:  0.9955568367685858
Precision score:  0.9936224489795918
Recall score:  0.9220026038584448
F1 score:  0.9564736939038616
Confusion matrix:
[[151072     50]
 [   659   7790]]
Logarithmic Loss:  0.025778977636169677
ROC AUC score:  0.9958348230189851


fitting for label threat
Accuracy score:  0.9997681282939882
Precision score:  1.0
Recall score:  0.9225941422594143
F1 score:  0.959738

In [63]:
# on test data
predictions_chain_test = classifier_chain.predict(selected_test[features])
# predict probability
predictions_proba_chain_test = classifier_chain.predict_proba(selected_test[features])

In [65]:
get_overall_evaluation_score(selected_test[labels], predictions_chain_test)

Accuracy score:  0.8914157991809685
Precision score:  0.5956152052922644
Recall score:  0.6432611394675127
F1 score:  0.6032385885358982


### Random Forest Using BinaryRelevance

In [67]:
classifier_br = BinaryRelevance(RandomForestClassifier(random_state = 0,class_weight='balanced',max_depth = 18))
# train
classifier_br.fit(selected_train[features], selected_train[labels])
# predict
predictions = classifier_br.predict(selected_train[features])

In [68]:
# predict probability
predictions_proba = classifier_br.predict_proba(selected_train[features])

In [69]:
# model evaluation
get_overall_evaluation_score(selected_train[labels], predictions)

Accuracy score:  0.9562514491981626
Precision score:  0.8258382390105613
Recall score:  0.988688814177446
F1 score:  0.8852303415231727


In [70]:
# model evaluation on each label
i = 0
for label in labels:
    print(f'fitting for label {label}')
    get_evaluation_score_single_class(selected_train[label], predictions[:,i].toarray(), predictions_proba[:,i].toarray())
    i += 1
    print("\n")

fitting for label toxic
Accuracy score:  0.993845999586391
Precision score:  0.9560866794136392
Recall score:  0.9808421603243102
F1 score:  0.9683062225664859
Confusion matrix:
[[143588    689]
 [   293  15001]]
Logarithmic Loss:  0.058609929692229515
ROC AUC score:  0.9994970298817809


fitting for label severe_toxic
Accuracy score:  0.9833553715900759
Precision score:  0.37520583392143025
Recall score:  1.0
F1 score:  0.5456722545330139
Confusion matrix:
[[155320   2656]
 [     0   1595]]
Logarithmic Loss:  0.03981942749854871
ROC AUC score:  0.9949235711848932


fitting for label obscene
Accuracy score:  0.9898415125555395
Precision score:  0.8441532258064516
Recall score:  0.9911232098473192
F1 score:  0.9117534977407589
Confusion matrix:
[[149576   1546]
 [    75   8374]]
Logarithmic Loss:  0.03730667399118395
ROC AUC score:  0.9983157234135245


fitting for label threat
Accuracy score:  0.9914896817090825
Precision score:  0.26034858387799564
Recall score:  1.0
F1 score:  0.4131

### Random Forest Using OneVsRest

In [72]:
for label in labels:
    print(f'fitting for label {label}')
    classifier_rf_ovr = RandomForestClassifier(random_state = 0,class_weight='balanced',max_depth = 18)
    classifier_rf_ovr.fit(selected_train[features], selected_train[label])
    predictions_rf_ovr = classifier_rf_ovr.predict(selected_train[features])
    predictions_proba_rf_ovr = classifier_rf_ovr.predict_proba(selected_train[features])
    print('Model Performance on training set:')
    get_evaluation_score_single_class(selected_train[label], predictions_rf_ovr, predictions_proba_rf_ovr[:, 1])
    
    predictions_rf_ovr_test = classifier_rf_ovr.predict(selected_test[features])
    predictions_proba_rf_ovr_test = classifier_rf_ovr.predict_proba(selected_test[features])
    print('Model Performance on test set:')
    get_evaluation_score_single_class(selected_test[label], predictions_rf_ovr_test, predictions_proba_rf_ovr_test[:, 1])

fitting for label toxic
Model Performance on training set:
Accuracy score:  0.993845999586391
Precision score:  0.9560866794136392
Recall score:  0.9808421603243102
F1 score:  0.9683062225664859
Confusion matrix:
[[143588    689]
 [   293  15001]]
Logarithmic Loss:  0.058609929692229515
ROC AUC score:  0.9994970298817809
Model Performance on test set:
Accuracy score:  0.9274594391822188
Precision score:  0.5999172527927182
Recall score:  0.7142857142857143
F1 score:  0.6521250281088374
Confusion matrix:
[[54987  2901]
 [ 1740  4350]]
Logarithmic Loss:  0.18532241031360303
ROC AUC score:  0.9478208188214192
fitting for label severe_toxic
Model Performance on training set:
Accuracy score:  0.9833553715900759
Precision score:  0.37520583392143025
Recall score:  1.0
F1 score:  0.5456722545330139
Confusion matrix:
[[155320   2656]
 [     0   1595]]
Logarithmic Loss:  0.03981942749854871
ROC AUC score:  0.9949235711848932
Model Performance on test set:
Accuracy score:  0.9779924348994967
Pre

## Oversampled Data

In [73]:
oversampled_train = pd.read_csv('Data/train_oversampled.csv')

In [74]:
print(oversampled_train.shape)
oversampled_train.head()

(421989, 49)


,toxic,obscene,insult,none,min_length_scaled,num_words_vs_length,exclamation_marks_vs_length,num_unique_words_scaled,verbs_vs_length,num_uppercase_scaled,...,139,143,156,157,170,198,clean_text,threat,identity_hate,severe_toxic
0,0,0,0,1,0.009393,0.181132,0.000000,0.106329,0.041509,0.003426,...,-0.109984,0.089084,0.012651,-0.019501,-0.091946,-0.113697,explanation why the edits made under my userna...,0,0,0
1,0,0,0,1,0.000723,0.160714,0.008929,0.043038,0.026786,0.001612,...,0.000094,0.021135,0.039308,-0.067295,-0.079366,-0.044959,d'aww! he matches this background colour i am ...,0,0,0
2,0,0,0,1,0.007225,0.188841,0.000000,0.091139,0.038627,0.000806,...,-0.146134,0.109907,0.036539,-0.053628,0.027530,-0.211230,"hey man, i am really not trying to edit war. i...",0,0,0
3,0,0,0,1,0.007948,0.175719,0.000000,0.141772,0.036741,0.002217,...,-0.214832,0.177560,0.007675,-0.087473,0.106848,-0.015987,""" more i cannot make any real suggestions on i...",0,0,0
4,0,0,0,1,0.003613,0.208955,0.000000,0.027848,0.059701,0.000403,...,-0.221076,0.163578,-0.068310,-0.031184,-0.017280,0.038566,"you, sir, are my hero. any chance you remember...",0,0,0


In [78]:
# logistic regression
classifier_over = MultiOutputClassifier(LogisticRegression(max_iter=10000))
# train
classifier_over.fit(oversampled_train[features], oversampled_train[labels])
# predict
predictions_over = classifier_over.predict(oversampled_train[features])

In [79]:
# predict probability
predictions_proba_log_ovr_over = classifier_over.predict_proba(oversampled_train[features])

In [80]:
# model evaluation overall score
get_overall_evaluation_score(oversampled_train[labels], predictions_over)

Accuracy score:  0.6146108073907139
Precision score:  0.9004191406353675
Recall score:  0.91221551936062
F1 score:  0.9061222418446149
